Tweet Intake

In [ ]:
#class override- apparently this is SOP for tweepy
class Listener(tweepy.StreamListener):
    
    def on_status(self,status):
        is_retweet = False
        #rt flag, start with false as default
        retweet_text= ""
        if hasattr(status, "retweeted_status"):
            is_retweet=True
            try:
                retweet_text = status.retweeted_status.extended_tweet["full_text"]
            except:
                retweet_text = status.retweeted_status.text
        
        #handles 140+ character tweets
        if hasattr(status, "extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text
        
        
        quoted_text =""
        if hasattr(status, "quoted_status"):
            #check if the QT was 140+ char
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet['full_text']
            else:
                quoted_text = status.quoted_status.text
                
        #some minor text-pre-processing to remove newlines and commas incase those become relevant
        remove = [',','\n']
        for i in remove:
            text = text.replace(i," ")
            quoted_text = quoted_text.replace(i," ")
            retweet_text = retweet_text.replace(i," ")
            
        with open("nlpdata.csv","a",encoding='utf-8') as f:
             f.write("%s,%s,%s,%s,%s,%s\n"%(status.user.screen_name,is_retweet,status.created_at,text,quoted_text,retweet_text))
        #db.processTest.insert({'handle':status.user.screen_name,'RT TF':is_retweet,'time':status.created_at,'tweet':text,'qt':quoted_text,'rt':retweet_text,'in_keyword_db':0})
    def on_error(self,status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()
    
    #def on_data(self,data):
        #print(data)

In [ ]:
streamListener = Listener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener, tweet_mode='extended')
with open('nlpdata.csv',"w",encoding='utf-8') as f:
    f.write("handle,is_rt,date,tweet,qt,rt")
tags = ["biden",'joe biden'] 
stream.filter(languages=["en"], track=tags)

Pre-processing functions

In [ ]:
from nltk.stem.snowball import SnowballStemmer
import string

#Create the function that performs the text conversion
sn = SnowballStemmer('english')

def clean_text(text, sn=sn):
    #sort of handles links, not 100% of the time, though
    link_re = re.compile('(?:(?:https?|ftp|file):\/\/|www\.|ftp\.)(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[-A-Z0-9+&@#\/%=~_|$?!:,.])*(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[A-Z0-9+&@#\/%=~_|$])')
    text=link_re.sub(' ',text)
    #special case of getting rid of RT info
    rt_re = re.compile('^RT @[a-zA-Z0-9]+:')
    text = rt_re.sub(' ',text)
    #removing other @ instances
    ats_re = re.compile('^@[a-zA-Z0-9]+')
    text = ats_re.sub(' ',text)
    #remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    #removing punctuation
    punc_re = re.compile('[%s]' % re.escape(string.punctuation + '£'))
    text = punc_re.sub(' ', ' '+text.lower()+' ') # Pad with spaces for easier stopword removal
    # Remove numbers
    num_re = re.compile('(\\d+)')
    text = num_re.sub(' ', text)
    # Remove alphanumerical words
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    text = alpha_num_re.sub(' ', text)
    # Stemming
    text = sn.stem(text)
    # Regex for multiple spaces
    spaces_re = re.compile('\s+')
    text = spaces_re.sub(' ', text.strip())

    return text

def removeSpecials(text):
    #for removal of special case strings
    hill_re = re.compile('thehill[a-zA-Z]*')
    text = hill_re.sub(' ',text)
    return text

WordCloud function. Takes in the corpus as a series and outputs wordcloud.

In [ ]:
def create_wordcloud(wordcloud_series):
    df_text = ' '.join(str(wordcloud_series).lower() for i in wordcloud_series)
    wordcloud = WordCloud(stopwords = None,
                      collocations=True).generate(df_text)
    plt.imshow(wordcloud, interpolation='bilInear')
    plt.axis('off')
    plt.show()
    
    
create_wordcloud(tweets)

VADER 'scoring':

In [ ]:
analyzer = SentimentIntensityAnalyzer()
summed=0
count=0
for sentence in tweets.tolist():
    vs = analyzer.polarity_scores(str(sentence))
    #print("{:-<65} {}".format(sentence, str(vs)))
    summed+=vs['compound']
    count+=1
print(summed)
print(summed/count)

For the VADER scoring, basically intook the Trump and Biden sets separately to get a score on sentiment towards those topics individually

Example CorEx topic modeling workflow:

In [ ]:
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english',
                             binary=True)

doc_word = vectorizer.fit_transform(tweets)
words = list(np.asarray(vectorizer.get_feature_names()))
topic_model = ct.Corex(n_hidden=15, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=tweets)
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))
#get 'baseline' topics, refine from there

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(range(topic_model.tcs.shape[0]), topic_model.tcs, color='#4e79a7', width=0.5)
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16);

In [ ]:
topic_model.tc
#snag TC of topics
#from here workflow is to identify coherent topics from those created by the 'baseline' and use them as anchor words

Lastly, Scattertext creation

In [ ]:
df1 = pd.read_csv("biden/bidenComp2.csv",error_bad_lines=False,quoting=3)
df2 = pd.read_csv("trump/trumpComp2.csv",error_bad_lines=False)
df1['label']='biden' #label the two sets separately
df2['label']='trump'
tweets=df1[['tweet','label']]
tweets2=df2[['tweet','label']]
full = pd.concat([tweets,tweets2],ignore_index=True)
full['tweet'] =full['tweet'].apply(str)
full['tweet']=full['tweet'].apply(clean_text)


In [ ]:
corpus = st.CorpusFromPandas(full,
                             category_col='label',
                             text_col='tweet',
                             nlp=st.whitespace_nlp_with_sentences
                            ).build()

In [ ]:
html = st.produce_scattertext_explorer(
        corpus,
        category='biden',
        category_name='Biden',
        not_category_name='Trump',
        minimum_term_frequency=10,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        )

In [ ]:
open('nlpScattertext.html', 'wb').write(html.encode('utf-8'));

Of particular note on the scattertext- it seems to fall apart pretty quickly once you get above ~50,000 documents or so.